In [10]:
from langchain_google_vertexai import ChatVertexAI
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate
)

import os
import json
import random

In [3]:
PATH = "..\\src\\data\\komus\\dataset.json"

with open(PATH, "r", encoding="UTF-8") as file:
    data = json.load(file)

len(data)

83755

In [6]:
PROJECT_ID = 'axial-chemist-425510-p2'
LOCATION = "europe-west4"

# Укажите путь к вашему JSON-файлу с кредами
SERVICE_ACCOUNT_FILE = "../secrets/axial-chemist-425510-p2-b4eb1d622fbe.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = SERVICE_ACCOUNT_FILE


llm_params = {
    'model_name': 'gemini-2.0-flash-001',
    'project': PROJECT_ID,
    'location': LOCATION
}

llm = ChatVertexAI(**llm_params)

In [7]:
SYSTEM_PROMPT = """
Perform an augmentation of the product position based on the given examples in a similar style.
Examples:
{examples}

The output should consist of the product item that was augmented.

**Do not generate any text before and after result.**
""".strip()

USER_PROMPT = """Give an answer to this position item {problem_title}"""

In [8]:
def create_model_prompts(system_prompt: str,
                         user_prompt: str) -> ChatPromptTemplate:
    system_prompt = SystemMessagePromptTemplate.from_template(system_prompt)
    user_prompt = HumanMessagePromptTemplate.from_template(user_prompt)
    chat_prompt = ChatPromptTemplate.from_messages(
        [system_prompt,
         user_prompt]
    )
    return chat_prompt


prompt = create_model_prompts(SYSTEM_PROMPT, USER_PROMPT)


In [9]:
examples = [
    {
        "original_item": "4723020140 КРЫШКА ГЛ.ТОРМ ЦИЛ Toyota",
        "clean_item": "Крышка главного тормозного цилиндра для автомобиля Toyota (4723020140)"
    },
    {
        "original_item": "АБР./К.НА ЛИП. P500, 15 ОТВ. 1ЕД=1ШТ SUNMIGHT",
        "clean_item": "Абразивный круг на липучке Sunmight P500, 15 отверстий? 1 шт."
    },
    {
        "original_item": "КРЫШКА Б.Б ОКРАСКА",
        "clean_item": "Окраска крышки бензобака"
    },
        {
        "original_item": "Вино игристое Вилла Чальдини Розе 2021, 0,75 л, 12%, Италия, Эмилия-Романия, Пр.И.В.И. с.р.л., розовое, брют",
        "clean_item": "Игристое вино розовое Villa Cialdini Brut Rose 2021, 0.75 л, 12%, Италия, Эмилия-Романия, Pr.I.V.I. s.r.l., брют "
    },
]

examples_str = "\n".join([f"""
{i+1})Product item: {product["clean_item"]}
Augmented item: {product['original_item']}
""".strip() for i, product in enumerate(examples)])
print(examples_str)

1)Product item: Крышка главного тормозного цилиндра для автомобиля Toyota (4723020140)
Augmented item: 4723020140 КРЫШКА ГЛ.ТОРМ ЦИЛ Toyota
2)Product item: Абразивный круг на липучке Sunmight P500, 15 отверстий? 1 шт.
Augmented item: АБР./К.НА ЛИП. P500, 15 ОТВ. 1ЕД=1ШТ SUNMIGHT
3)Product item: Окраска крышки бензобака
Augmented item: КРЫШКА Б.Б ОКРАСКА
4)Product item: Игристое вино розовое Villa Cialdini Brut Rose 2021, 0.75 л, 12%, Италия, Эмилия-Романия, Pr.I.V.I. s.r.l., брют 
Augmented item: Вино игристое Вилла Чальдини Розе 2021, 0,75 л, 12%, Италия, Эмилия-Романия, Пр.И.В.И. с.р.л., розовое, брют


## Случайный выбор из подборки

In [11]:
idx = random.choices(list(range(len(data))), k = 10)
idx

[22009, 6842, 3491, 11635, 54735, 66683, 64314, 31085, 17752, 8034]

## Запуск модели

In [13]:
similarity_chain = prompt | llm

for id in idx:
    for i in range(2):
        product = data[id]

        res = similarity_chain.invoke({
            "examples": examples_str,
            "problem_title": product["title"],
        })

        print(product["title"])
        # print(json.dumps(product["attributes"], ensure_ascii=False, indent=4))
        print(f"Ответ: {res.content}")
    

Картридж лазерный Хеrох 106R01277 черный оригинальный (двойная упаковка)
Ответ: Картридж лазерный Xerox 106R01277, черный, оригинал (двойная упаковка)

Картридж лазерный Хеrох 106R01277 черный оригинальный (двойная упаковка)
Ответ: Картридж лазерный Xerox 106R01277, черный, оригинал, двойная упаковка

Чай травяной Imperial Tea Collection 20 пакетиков (имбирь, анис, корица)
Ответ: Чай травяной Империал Ти Коллекшн (имбирь, анис, корица) 20 пакетиков

Чай травяной Imperial Tea Collection 20 пакетиков (имбирь, анис, корица)
Ответ: Чай травяной Империал ТИ коллекшн 20 пак (Имбирь, Анис, Корица)

Батарея для ИБП ExeGate HR 12-9 12 В 9 Ач (EP129860RUS)
Ответ: БАТАРЕЯ ДЛЯ ИБП 12В 9АЧ EXEGATE HR 12-9

Батарея для ИБП ExeGate HR 12-9 12 В 9 Ач (EP129860RUS)
Ответ: БАТАРЕЯ ДЛЯ ИБП 12В 9АЧ EXEGATE HR 12-9 (EP129860RUS)

Рабочая станция Avance 6МБН.6277 двухместная (венге/алюминий матовый, 1000х1435х750 мм)
Ответ: РАБ.СТАНЦИЯ AVANCE 6МБН.6277 2-Х МЕСТН. (ВЕНГЕ/АЛЮМ.МАТ., 1000Х1435Х750 ММ)

Рабочая